In [1]:
!pip3 install -U datapane

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.6/143.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 85.0 MB/s eta 0:00:00
  Created wheel for micawber: filename=micawber-0.5.6-py3-none-any.whl size=22457 sha256=99e60bdbb4d0b1d8c78f0334c37cbff7d1280db22f5374d08b830e2324470932
  Stored in directory: /root/.cache/pip/wheels/74/e3/a2/e7072ad68319f81a1aa33ab2724210e6c71b3e72e58dd0f847
Successfully built micawber
  Attempting uninstall: packaging
    Found existing installation: packaging 24.2
    Uninstalling packaging-24.2:
      Successfully uninstalled packaging-24.2
  Attempting uninstall: lxml
    Found existing installation: lxml 5.3.0
    Uninstalling lxml-5.3.0:
      Successfully uninstalled lxml-5.3.0
  Attempting uninstall: pandas
    Found existing i

In [2]:
import pandas as pd
import numpy as np
import plotly.figure_factory as ff
import plotly.graph_objs as go
import plotly.io as pio
import datapane as dp

In [3]:
symbols = ["ETH","BTC","SOL","BNB"]

In [4]:
data_dict = {}
for sym in symbols:
    data = pd.read_csv(f"/kaggle/input/crypto-currencies-daily-prices/{sym}.csv")
    data["date"] = pd.to_datetime(data["date"], format="%Y-%m-%d")
    data_dict[sym] = data

fig_list = []
for sym, data in data_dict.items():
    data["10-day MA"] = data["close"].rolling(window=10).mean()
    data["20-day MA"] = data["close"].rolling(window=20).mean()
    trace0 = go.Scatter(x=data.date, y=data.close, name=sym)
    trace1 = go.Scatter(x=data.date, y=data["10-day MA"], name="10-day MA")
    trace2 = go.Scatter(x=data.date, y=data["20-day MA"], name="20-day MA")
    fig = go.Figure([trace0, trace1, trace2])
    fig.update_layout(title={"text": "{sym} Price", "x": 0.5, "xanchor": "center"})
    fig_list.append(fig)

In [5]:
trace_list = []
for sym, data in data_dict.items():
    trace = go.Scatter(x=data.date, y=data.close, name=sym)
    trace_list.append(trace)
fig = go.Figure(trace_list)
fig.update_layout(
    dict(
        title=dict({"text": "TVL Top4 Price Comparision", "x": 0.5, "xanchor": "center"}),
        xaxis=dict(
            rangeselector=dict(
                buttons=list(
                    [
                        dict(count=1, label="1m", step="month", stepmode="backward"),
                        dict(count=3, label="3m", step="month", stepmode="backward"),
                        dict(count=6, label="6m", step="month", stepmode="backward"),
                        dict(count=12, label="12m", step="month", stepmode="backward"),
                    ]
                )
            ),
            rangeslider=dict(visible=True),
            type="date",
        ),
    )
)
fig_list.append(fig)


In [6]:
trace_list = []
for sym, data in data_dict.items():
    trace = go.Scatterpolar(
        r=[data["close"].mean(), data["open"].mean(), data["low"].min(), data["high"].max()],
        theta=["Close", "Open", "Low", "High"],
        name=sym,
        fill="toself",
    )
    trace_list.append(trace)
fig = go.Figure(trace_list)
fig.update_layout(
    go.Layout(
        polar=dict(radialaxis=dict(visible=True)),
        title=dict({"text": "TVL Top4 Price Comparision (Radar Chart)", "x": 0.5, "xanchor": "center"}),
    )
)
fig_list.append(fig)

eth = data_dict["ETH"][["open", "close"]]
eth["index"] = np.arange(len(eth))
fig_eth = go.Figure(
    ff.create_scatterplotmatrix(
        eth,
        diag="box",
        index="index",
        size=3,
        height=600,
        width=1150,
        colormap="RdBu",
        title={"text": "ETH Price (Scatterplot Matrix)", "x": 0.5, "xanchor": "center"},
    )
)

In [7]:
for fig in fig_list:
    fig.show()
fig_eth.show()